In [186]:
import requests, httpx
import json
from urllib.parse import urlencode, quote, urlsplit, parse_qs, parse_qsl, unquote, unquote_plus
from barely_json import parse
import jmespath
headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'x-asbd-id': '129477',
    'x-ig-app-id': '936619743392459',
    'cookie': '''csrftoken=cpBvgDRGLIQ4j3RgScGvH24w9R6cQffX; ds_user_id=18519728814; ps_n=1; ps_l=1; mid=Zl0FuQALAAHWLneH1O51KnPmZbQO; ig_did=8F185B9B-0018-4D2C-9C8B-5861E8826960; dpr=0.8640000224113464; datr=fid7ZoVfi9JAdPRsF4AUt5H9; shbid="19508\05418519728814\0541751227322:01f71e1583bc74deb0cf35aadeef14cbf22516af31cc981c85d6a595394c0fc9c9797cbb"; shbts="1719691322\05418519728814\0541751227322:01f797a9ebf1355c2fe47a789c2fca5f528433508e5c351bbb67dc69bcb3920b918ea3d3"; sessionid=18519728814%3A8CptQEBMl6lstb%3A29%3AAYfPHfaU0TD5O2F2otwE2BteTZDIh3lS8N5qbid1QQ; rur="RVA\05418519728814\0541751409798:01f7914db0d124d7da5a7af3e8dbeca4c2fc69e72380836ba0862459726e5ae2d6f8640f"; wd=1582x268'''
    }

In [190]:
def prepare_endpoint(variables):
    '''Return an api endpoint for data about the first 12 posts on a user page'''
    json_str = json.dumps(vars_dict)
    return f"https://www.instagram.com/graphql/query/?query_hash=e769aa130647d2354c40ea6a439bfc08&variables={quote(json_str)}"

variables = {
  "id": "1067259270",
  "first": 12,
  "after": None
}
prepare_endpoint(variables)

'https://www.instagram.com/graphql/query/?query_hash=e769aa130647d2354c40ea6a439bfc08&variables=%7B%22id%22%3A%201067259270%2C%20%22first%22%3A%2012%7D'

In [125]:
def change_variables(url, first=12):
    ''' ! Use prepare_endpoint() instead ! \n
    Parse an already existing all-user posts endpoint, allows manipulating it and 
    returns new endpoint 
    '''
    url_parts = urlparse(url)
    query = url_parts.query
    variables = parse_qs(query)['variables'][0]
    vars_dict = parse(variables)
    vars_dict['first'] = first
    vars_dict['id'] = 1067259270
    json_str = json.dumps(vars_dict)
    return f"https://www.instagram.com/graphql/query/?query_hash=e769aa130647d2354c40ea6a439bfc08&variables={quote(json_str)}"
endpoint = change_variables(endpoint)

In [126]:
resp = requests.get(endpoint)
resp.status_code

200

In [185]:
# Testing comments in posts
data = resp.json()['data']['user']['edge_owner_to_timeline_media']['edges'][0]['node']
jmespath.search('''{
    shortcode: shortcode,
    comments: edge_media_to_comment.edges[].node[].text,
    page
    }''', data)

{'shortcode': 'C8senjXOVkv',
 'comments': ['👏', 'Love her!', 'Fix my business profile!!']}

In [178]:
# Check keys for each post
resp.json()['data']['user']['edge_owner_to_timeline_media']['edges'][0]['node'].keys()

dict_keys(['__typename', 'id', 'dimensions', 'display_url', 'display_resources', 'is_video', 'tracking_token', 'edge_media_to_tagged_user', 'accessibility_caption', 'edge_media_to_caption', 'shortcode', 'edge_media_to_comment', 'edge_media_to_sponsor_user', 'comments_disabled', 'taken_at_timestamp', 'edge_media_preview_like', 'gating_info', 'fact_check_overall_rating', 'fact_check_information', 'media_preview', 'owner', 'location', 'viewer_has_liked', 'viewer_has_saved', 'viewer_has_saved_to_collection', 'viewer_in_photo_of_you', 'viewer_can_reshare', 'thumbnail_src', 'thumbnail_resources'])

In [180]:
# Check if changing returned posts number will break the api (seems like it's working)
len(resp.json()['data']['user']['edge_owner_to_timeline_media']['edges'])

20

In [189]:
# Checking what's end_cursor, used for page offset
posts = resp.json()['data']['user']['edge_owner_to_timeline_media']
jmespath.search('''
    page_info
    ''', posts)

{'has_next_page': True,
 'end_cursor': 'QVFENlBNVWZnQ3k2ZlNjQzdrUG5ubFI4cVhuOGJEYWxIVWpNUDhyTC1keFFDcWlCQU4yZmZuVlk2OWR3Uk9WR2FPN1pCbHJkbkRmV2ViU3hzTzVscnZLdg=='}

In [ ]:
# Currently used API from curl to py; To be checked for differences...
import requests

cookies = {
    'csrftoken': 'cpBvgDRGLIQ4j3RgScGvH24w9R6cQffX',
    'ds_user_id': '18519728814',
    'ps_n': '1',
    'ps_l': '1',
    'mid': 'Zl0FuQALAAHWLneH1O51KnPmZbQO',
    'ig_did': '8F185B9B-0018-4D2C-9C8B-5861E8826960',
    'dpr': '0.8640000224113464',
    'datr': 'fid7ZoVfi9JAdPRsF4AUt5H9',
    'shbid': '"19508\\05418519728814\\0541751540486:01f76e5af287c0ac4b0712ad9a199516ec3a212eab65151b15d328a48527f468f5d48c84"',
    'shbts': '"1720004486\\05418519728814\\0541751540486:01f756041ca2b44d4b4b77389ca6928566e32f7d821f7125f673e2985202b37c288ae188"',
    'sessionid': '18519728814%3A8CptQEBMl6lstb%3A29%3AAYdUd--zj1AvDDlxLONWLEyqmoeFwK0gn-IUlZcipcg',
    'rur': '"RVA\\05418519728814\\0541751543175:01f7df8e0d96f9c638821e7792fad32ba1a767e408602bb0dda7dcba5aecf9adb03b956c"',
    'wd': '1582x229',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # 'cookie': 'csrftoken=cpBvgDRGLIQ4j3RgScGvH24w9R6cQffX; ds_user_id=18519728814; ps_n=1; ps_l=1; mid=Zl0FuQALAAHWLneH1O51KnPmZbQO; ig_did=8F185B9B-0018-4D2C-9C8B-5861E8826960; dpr=0.8640000224113464; datr=fid7ZoVfi9JAdPRsF4AUt5H9; shbid="19508\\05418519728814\\0541751540486:01f76e5af287c0ac4b0712ad9a199516ec3a212eab65151b15d328a48527f468f5d48c84"; shbts="1720004486\\05418519728814\\0541751540486:01f756041ca2b44d4b4b77389ca6928566e32f7d821f7125f673e2985202b37c288ae188"; sessionid=18519728814%3A8CptQEBMl6lstb%3A29%3AAYdUd--zj1AvDDlxLONWLEyqmoeFwK0gn-IUlZcipcg; rur="RVA\\05418519728814\\0541751543175:01f7df8e0d96f9c638821e7792fad32ba1a767e408602bb0dda7dcba5aecf9adb03b956c"; wd=1582x229',
    'origin': 'https://www.instagram.com',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://www.instagram.com/p/C86aabgxXNC/?hl=en',
    'sec-ch-prefers-color-scheme': 'dark',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'sec-ch-ua-full-version-list': '"Not/A)Brand";v="8.0.0.0", "Chromium";v="126.0.6478.127", "Google Chrome";v="126.0.6478.127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '""',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"10.0.0"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'x-asbd-id': '129477',
    'x-bloks-version-id': '213e4f338be29ab2c08f8071c4feb979c6b2fe37d4124f56e5c4b00e51a21aaf',
    'x-csrftoken': 'cpBvgDRGLIQ4j3RgScGvH24w9R6cQffX',
    'x-fb-friendly-name': 'PolarisProfilePostsDirectQuery',
    'x-fb-lsd': 'DC7iNn2Z-NWJN8Zffof_av',
    'x-ig-app-id': '936619743392459',
}

data = {
    'av': '17841418504424966',
    'hl': 'en',
    '__d': 'www',
    '__user': '0',
    '__a': '1',
    '__req': '1d',
    '__hs': '19907.HYP:instagram_web_pkg.2.1..0.1',
    'dpr': '1',
    '__ccg': 'UNKNOWN',
    '__rev': '1014646830',
    '__s': 'kea2bd:lp0dbk:9l65ly',
    '__hsi': '7387363004652082787',
    '__dyn': '7xeUjG1mxu1syUbFp41twpUnwgU7SbzEdF8aUco2qwJxS0k24o0B-q1ew65xO0FE2awpUO0n24oaEnxO1ywOwv89k2C1Fwc60D87u3ifK0EUjwGzEaE2iwNwKwHw8Xxm16wUwtEvw4JwJCwLyES1TwTU9UaQ0Lo6-3u2WE5B08-269wr86C1mwPwUQp1yUb9UK6V8aUuxK3OqcyU-2K',
    '__csr': 'gPb2Q6n2ccPErPhAONyN4iBdOqcybVibvumZnivijpVBBChFp4AGUx4zu-x1kDUiAKlaWJki9WAChybGGAKi8CDJqGAFqCgR2F4jAKFp456VogVWCzG-aK5CWG8yoS8XGaF4DyqKdqzojK7ohnBAZ005oromwjk1xw8sw2yxnU1k4bw2Eo0tKV7gighyeNEsw9N0c6qeCgeYE8UrVLc0h1w2c8hw9rw6Qg5WlS0t2E5SE7q0EUW0TA1gz8G4kaw9Iw02nDw',
    '__comet_req': '7',
    'fb_dtsg': 'NAcO9iIY3ARgsccTS3Y_6oIM_uq6ccudpgU4KtfOtpnNwQbSNFJ76lg:17853667720085245:1683203180',
    'jazoest': '26353',
    'lsd': 'DC7iNn2Z-NWJN8Zffof_av',
    '__spin_r': '1014646830',
    '__spin_b': 'trunk',
    '__spin_t': '1720004483',
    'fb_api_caller_class': 'RelayModern',
    'fb_api_req_friendly_name': 'PolarisProfilePostsDirectQuery',
    'variables': '{"data":{"count":12,"include_relationship_info":true,"latest_besties_reel_media":true,"latest_reel_media":true},"username":"fcbarcelona","__relay_internal__pv__PolarisFeedShareMenurelayprovider":false}',
    'server_timestamps': 'true',
    'doc_id': '8721952884485930',
}

response = requests.post('https://www.instagram.com/graphql/query', cookies=cookies, headers=headers, data=data)